## Example of generating moving object observations, using `MoObs` class.

This is the class that should be used to generate moving object observations. (it's beta, but it's better than early prototype). 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from moo import MoObs

Expanded version (I want to test the timing). For a very compact way to generate observations, see the `runMoObs` function in moo.py.

Set up and read the orbits.

In [13]:
orbitfile = 'GreenstreetPHAs.des'
#orbitfile = 'pha20141031.des'
moo = MoObs()
moo.readOrbits(orbitfile)

In [14]:
# See what we've got in the orbit data.
print moo.orbits.dtype.names
print moo.orbits.epoch

('objId', 'q', 'e', 'inc', 'node', 'argPeri', 'tPeri', 'epoch', 'H', 'g', 'a', 'meanAnom')
[49353 49353 49353 ..., 49353 49353 49353]


Generate ephemerides for ten years, with spacing of 2 hours, then generate interpolation functions.

In [16]:
moo.setTimes(timestep = 2./24., ndays=365*10, timestart = 49353.0)
moo.setupOorb()

In [18]:
%%timeit
for sso in moo.orbits[0:1]:
    ephs = moo.generateEphs(sso)
    interpfuncs = moo.interpolateEphs(ephs)

1 loops, best of 3: 7.3 s per loop


Getting 21.4s/loop (3 objects) and 24.8s/loop (3 objects). This is comparable to previous estimates, where it was 8s/object for ten years. For objects more distant than NEOs, we can increase the timestep. 

As seen in the [Ephemeris Generation](./Ephemeris Generation.ipynb) notebook, the ephemeris accuracy for 2 hour timespacing + linear interpolation for NEOs is generally better than 3e-5 degrees (0.1") in position, and 8.3e-6 ("/s) in velocity. The solar elongation calculation for some NEOs seems to be faulty -- or at least, not agree with opsim (and I doubt opsim is really observing things with very low solar elongation of <20deg). That needs a little more investigation, but for now - use opsim's solar elongation values.

---

Add in actual simdata series of observations and matching moving object to the fov.

In [23]:
from lsst.sims.maf.db import OpsimDatabase

opsdb = OpsimDatabase('/Users/lynnej/opsim/db/enigma_1189_sqlite.db')

# What columns from opsim do you want to have available when doing later 'metric' generation?
dbcols = ['expMJD', 'night', 'fieldRA', 'fieldDec', 'rotSkyPos', 'filter',
          'finSeeing', 'fiveSigmaDepth', 'visitExpTime', 'solarElong']
simdata = opsdb.fetchMetricData(dbcols, sqlconstraint='')

/Users/lynnej/anaconda/lib/python2.7/site-packages/sqlalchemy/sql/elements.py:3758: SAWarning: Textual SQL expression '' should be explicitly declared as text('') (this warning may be suppressed after 10 occurrences)
  {"expr": util.ellipses_string(element)})


In [38]:
%%timeit
outfileName = 'test_out.txt'
del moo.outfile 
useCamera=True
# Limit this example to only calculating positions for the first few objects.
for sso in moo.orbits[0:3]:
    ephs = moo.generateEphs(sso)
    interpfuncs = moo.interpolateEphs(ephs)
    # This is how you change the matching against the FOV. 
    #  If useCamera=True, then rFov is irrelevant - the silicon footprint is used.
    #  If useCamera=False, then rFov is used instead, as a circular footprint. 
    idxObs = moo.ssoInFov(interpfuncs, simdata, rFov=np.radians(1.75), useCamera=useCamera)
    moo.writeObs(sso['objId'], interpfuncs, simdata, idxObs, outfileName=outfileName)

1 loops, best of 3: 30.5 s per loop


With the camera on, writing to disk (desktop = SSD), this took 30s for 3 objects. With camera off, this took 24s for 3 objects. 

In [34]:
# See what this output file looks like. 
# This file stores the moving object observations - even if the object would have been too faint!
!head test_out.txt

#objId delta ra dec magV time dradt ddecdt phase solarelon velocity obsHistID expMJD night fieldRA fieldDec rotSkyPos filter finSeeing fiveSigmaDepth visitExpTime solarElong dmagColor dmagTrail dmagDetect 
1 16.7655846845 138.743455494 16.4421927808 371.931325489 49360.244826 -0.0296064149448 0.00930690723584 1.51658958398 151.662413843 0.0310348153406 3320 49360.244826 7 2.426603 0.261673 1.787356 g 0.814579 24.641303 30.0 150.868114 0.302320621737 0.00088603843245 0.000516878877234 
1 16.7655211962 138.743151539 16.4422844454 371.931268622 49360.254681 -0.0296218314226 0.00929532713063 1.51607368149 151.672882918 0.0310460399797 3342 49360.254681 7 2.426603 0.261673 1.781385 g 0.794688 24.668984 30.0 150.878281 0.302320621737 0.000930346881922 0.00054345720931 
1 16.7649565096 138.740406268 16.4431050476 371.930756405 49360.343518 -0.0296186423499 0.00918606339935 1.51141991249 151.767095804 0.0310104469287 3537 49360.343518 7 2.426603 0.261673 5.014768 g 0.725173 24.632871 30.0 150.